In [1]:
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords # get stopwords to remove
import re # regular expression
from gensim.models import doc2vec, Word2Vec # for word embeddings
from gensim.utils import simple_preprocess # to tokenize automatically
from sklearn.model_selection import train_test_split, KFold # for test-train split & cross validation
from sklearn.preprocessing import MultiLabelBinarizer # to convert to a format that can do multi-label classification
import random
import keras # for nn
import tensorflow as tf # for nn & new loss
import keras.backend.tensorflow_backend as tfb # for nn $ new loss
from sklearn.metrics import precision_score, accuracy_score, roc_auc_score, f1_score # for metrics
import collections

stop = stopwords.words('english')
# Ensure reproducibility
seed = 561
np.random.seed(seed)
random.seed(seed)
tf.set_random_seed(seed)

unable to import 'smart_open.gcs', disabling that module
Using TensorFlow backend.


In [2]:
# Constants
vec_dim = 100 # how big the word embeddings are

# Multi-label classification

Classify the reasons (violations) behind each docket_num (document).

# Read in data

In [3]:
raw_text = pd.read_csv('./data/clean_mea_text.csv') # this holds the raw text
reasons = pd.read_csv('./data/mea_reasons_filtered.csv') # these are our target classifications

In [4]:
print(raw_text.head())
print(raw_text.shape)

         date docket_num                                               text  \
0  2009-11-18     09_160  STATE OF NORTH CAROLINA\nWAKE COUNTY\nIN A MAT...   
1  2009-11-18     09_164  STATE OF NORTH CAROLINA\nWAKE COUNTY\nIN A MAT...   
2  2009-10-16    09_142B  OAH File No. 10 COB 2895\nSTATE OF NORTH CAROL...   
3  2009-09-09     09_081  STATE OF NORTH CAROLINA\nWAKE COUNTY\nIN A MAT...   
4  2009-08-24     09_070  STATE OF NORTH CAROLINA\nWAKE COUNTY\nIN A MAT...   

   year  month  day  
0  2009     11   18  
1  2009     11   18  
2  2009     10   16  
3  2009      9    9  
4  2009      8   24  
(177, 6)


In [5]:
print(reasons.head())
print(reasons.shape)

  docket_num        date                                        reason
0     09_160  11/18/2009                    Conspiracy to commit fraud
1     09_164  11/18/2009                    Conspiracy to commit fraud
2    09_142B  10/16/2009                     Allowed unlawful activity
3     09_081    9/9/2009  Falsification and misrepresentation of loans
4     09_070   8/24/2009                       Retained borrower funds
(375, 3)


In [6]:
# There is a bit of data mismatch, so filter both dfs for text that appears in both
bothdocs = set(raw_text.docket_num.values).intersection(reasons.docket_num.values)
raw_text = raw_text[raw_text.docket_num.isin(bothdocs)]
reasons = reasons[reasons.docket_num.isin(bothdocs)]
print(raw_text.shape)
print(reasons.shape)

# Also need to convert datatypes to prevent type mismatch
raw_text['text'] = raw_text['text'].astype(str)

(169, 6)
(359, 3)


In [7]:
# Remove irrelevant/unhelpful labels
toremove = ['Unknown', 
           'Other', 
           'Did not appear for the Hearing', 
           'Other failure to disclose', 
           'Unknown failure to disclose', 
           'Allowed falsification and misrepresentation of loans',
           'Circumvented the requirements that a branch manager of a licensed mortgage broker have at least three years experience',
           "Did not verify or make a reasonable effort to verify the borrower's information",
           'Employed simultaneously by more than one affiliated mortgage banker or mortgage broker',
           'Engaged in fraud',
           'Failure to disclose charges',
           'Violated NC Securities Act',
           'Withdrew appeal',
           'Unsatisfactory credit']
reasons = reasons[~reasons.reason.isin(toremove)]

In [8]:
# Since we want to do multi-label classification, binarize outputs
# First, need to aggregate reason by docket_num
reasonsls = reasons.groupby('docket_num')['reason'].apply(set).reset_index(name='reason')

mlb = MultiLabelBinarizer()
classesbin = mlb.fit_transform(reasonsls.reason.values)
classesbin = pd.DataFrame(classesbin)
classesbin.columns = mlb.classes_

reasonsls = pd.concat([reasonsls, classesbin], axis=1)

In [9]:
# Let's combine the input and output datasets for easier handling
merged = raw_text.merge(reasonsls)

In [10]:
print(merged.shape)

(153, 25)


# Preprocessing

In [11]:
# Params:
#    df - dataframe with column 'text' to be tokenized
#    tokens_only - to train the doc2vec model, we’ll need to 
#        associate a tag/number with each document of the training corpus. 
#        tokens_only=True means don't associate anything
def tokenize(df, tokens_only=False):
    tokens = df['text'].apply(lambda x: simple_preprocess(x, deacc=True, max_len=20)) # max_len=20 just in case there are important words 15 chars long)
    if tokens_only:
        return tokens
    else:
        # For training data, add tags -- notice it is just an index number
        return [doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(tokens)]

# Vectorize

We use the Continuous Bag of Words (CBOW) model to create our word embeddings to be used in our ML models.

In [12]:
# Remove extra space
merged['text'] = merged['text'].apply(lambda x: re.sub(r'\s+', ' ', x))
# Tokenize
corpus = tokenize(merged)

docmodel = doc2vec.Doc2Vec(vector_size=vec_dim,min_count=1, epochs=40) # min_count=1 because we're not sure if relevant words occur multiple times
docmodel.build_vocab(corpus)
# Train
docmodel.train(corpus, total_examples=docmodel.corpus_count, epochs=docmodel.epochs)

In [13]:
# Let's also use word2vec and averaging to compare to doc2vec
# Perhaps in this setting, the context of words isn't important
textls = tokenize(merged, tokens_only=True)
wordmodel = Word2Vec(textls, min_count=1)
w2v = dict(zip(wordmodel.wv.index2word, wordmodel.wv.syn0))

C:\Users\Phoenix\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """


In [14]:
# Params:
#    w2v - dictionary of words to vectors
#    text - list of tokenized words to convert to vector
# Returns a vector resulting from the average of all present words in text
def average_vectors(w2v, text):
    num_count = collections.Counter(text) # words and their counts
    return np.mean([w2v[word]*count for word, count in num_count.items()], axis=0)

In [15]:
docvecs_avg = [average_vectors(w2v, corpus[i].words) for i in range(len(corpus))]

## Assess Doc2Vec CBOW model

In [16]:
ranks = []
second_ranks = []
docvecs = []
for doc_id in range(len(corpus)):
    inferred_vector = docmodel.infer_vector(corpus[doc_id].words)
    sims = docmodel.docvecs.most_similar([inferred_vector], topn=len(docmodel.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
    
    docvecs.append(inferred_vector)

In [17]:
counter = collections.Counter(ranks)
print(counter)

Counter({0: 147, 1: 6})


## Add doc embeddings

In [18]:
merged = pd.concat([merged, pd.DataFrame(docvecs)], axis=1)
merged = pd.concat([merged, pd.DataFrame(docvecs_avg)], axis=1)

In [19]:
print(merged.columns.values)

['date' 'docket_num' 'text' 'year' 'month' 'day' 'reason'
 'Allowed unlawful activity'
 'Concealed material facts that were likely to influence a mortgagor to take a mortgage loan'
 'Conspiracy to commit fraud' 'Convicted of misdemeanor or felony'
 'Does not meet the financial responsibility requirements' 'Excess fees'
 'Failure to disclose civil or financial issue'
 'Failure to disclose criminal conviction'
 'Failure to file a correcting amendment'
 'Falsification and misrepresentation of loans'
 'Has outstanding tax liens' 'Identity issues'
 'Impermissible net-branching' 'Lacks a surety bond'
 'Mistreatment of employees' 'Retained borrower funds'
 'Uncooperative with OCOB' 'Unlicensed activity' 0 1 2 3 4 5 6 7 8 9 10 11
 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35
 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

# Train & Test Neural Network model

In [20]:
class_dim = len(mlb.classes_) # number of distinct classes
epochs = 25
batch_size = 10
k_folds = 5 # number of folds for cv
pweights = [15, 10, 5, 4, 3, 2, 1]  # multiplier for positive targets, needs to be tuned

pos_weight = pweights[0] # starting weight

## New loss

In [21]:
# From https://stackoverflow.com/questions/42158866/neural-network-for-multi-label-classification-with-large-number-of-classes-outpu/47313183#47313183
def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tf.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(targets=target,
                                                    logits=output,
                                                    pos_weight=pos_weight)
    return tf.reduce_mean(loss, axis=-1)

## New metric

In [22]:
# Function that returns the hamming score
# correctly predicted / number of labels
# Effectively acts as an accuracy metric in multilabel classification
def hamming_score(y_true, y_pred):
    return (y_pred == y_true).mean()

In [23]:
# Function that given predicted probabilities 
# and the true labels,
# calculates a bunch of metrics and returns them
def calc_metrics(y_true, y_prob):
    y_pred = np.copy(y_prob) # classes
    y_pred[y_pred>=0.5] = 1
    y_pred[y_pred<0.5] = 0

    # Metrics
    # average='micro' because we care a little more about global statistics
    ham_score = hamming_score(y_true, y_pred) # accuracy
    emr = accuracy_score(y_true, y_pred) # exact match ratio
    f1 = f1_score(y_true, y_pred, average='micro') # f1 -- care about false positives and false negatives
    prec = precision_score(y_true, y_pred, average='micro') # tp / (tp + fp) # care about false positives slightly more let's look at precision instead of both
    auc = roc_auc_score(y_true, y_prob, average='micro') # for ease, pretend the threshold should be the same for all classes
    metrics = [ham_score, emr, f1, prec, auc]
    
    return metrics

## Train & test

In [24]:
# Function that runs k_folds cross validation
# Note that x, y, and kf is set outside of this function
def run_cv(verbose=True):
    cv_scores = np.empty((k_folds, num_metrics))
    j = 0
    for train_index, test_index in kf.split(x):
        if verbose:
            print('{:d}th CV run'.format(j))
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Build model
        model = keras.models.Sequential() 
        model.add(keras.layers.Dense(128, input_dim=vec_dim, activation='relu'))
        model.add(keras.layers.Dense(64, activation='relu'))
        model.add(keras.layers.Dense(class_dim, activation='sigmoid')) # sigmoid so that the probability of one class is independent from the probability of another
        model.compile(loss=weighted_binary_crossentropy, 
                  optimizer='adam')
        # Fit model
        model.fit(X_train, 
                     y_train,
                     epochs=epochs,
                     batch_size=batch_size,
                     verbose=0)
        # Evaluate
        y_prob = model.predict(X_test) # probabilities
        metrics = calc_metrics(y_test.values, y_prob)
        
        if verbose:
            print(metrics)
        
        for k in range(num_metrics):
            cv_scores[j, k] = metrics[k]
            
        j = j + 1
        
        # To prevent slower performance
        tfb.clear_session()
    return cv_scores.mean(axis=0)

In [25]:
# Split into x and y
x = merged.iloc[:, 7+class_dim:7+class_dim+vec_dim]
y = merged.iloc[:, 7:7+class_dim]

In [26]:
# Cross validation
kf = KFold(n_splits=k_folds, shuffle=True)
num_metrics = 5 # number of metrics

cv_scores = np.empty((len(pweights),k_folds, num_metrics))
for i in range(len(pweights)):
    pos_weight = pweights[i]
    j = 0 # to index which cv it is
    for train_index, test_index in kf.split(x):
        print('pos_weight={:d}, {:d}th CV run'.format(pweights[i], j))
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Build model
        model = keras.models.Sequential() 
        model.add(keras.layers.Dense(128, input_dim=vec_dim, activation='relu'))
        model.add(keras.layers.Dense(64, activation='relu'))
        model.add(keras.layers.Dense(class_dim, activation='sigmoid')) # sigmoid so that the probability of one class is independent from the probability of another
        model.compile(loss=weighted_binary_crossentropy, 
                  optimizer='adam')
        # Fit model
        model.fit(X_train, 
                     y_train,
                     epochs=epochs,
                     batch_size=batch_size,
                     verbose=0)
        
        # Evaluate
        y_prob = model.predict(X_test) # probabilities
        metrics = calc_metrics(y_test.values, y_prob)
        
        print(metrics)
        
        for k in range(num_metrics):
            cv_scores[i, j, k] = metrics[k]
        j = j + 1
        
        # To prevent slower performance
        tfb.clear_session()

0th CV run
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[0.899641577060932, 0.16129032258064516, 0.5692307692307692, 0.5606060606060606, 0.8137651821862347]
1th CV run
[0.8870967741935484, 0.03225806451612903, 0.5401459854014599, 0.5068493150684932, 0.8600075910931173]
2th CV run
[0.8817204301075269, 0.0967741935483871, 0.547945205479452, 0.449438202247191, 0.8418601393703821]
3th CV run
[0.8870370370370371, 0.16666666666666666, 0.5196850393700787, 0.4852941176470588, 0.882554001198069]
4th CV run
[0.9092592592592592, 0.2, 0.5504587155963303, 0.5, 0.8563531318841181]


ValueError: setting an array element with a sequence.

In [26]:
cv_scores.mean(axis=1) # find average for each metric by pos_weight -- each index now corresponds to a pos_weight
# array([[0.88677419, 0.12430108, 0.52165945, 0.47706149, 0.84656246],
#        [0.88882915, 0.17010753, 0.50038028, 0.47980143, 0.83313419],
#        [0.90169654, 0.19010753, 0.51369912, 0.5431016 , 0.8435737 ],
#        [0.90663082, 0.18258065, 0.52858373, 0.57419763, 0.85126949],
#        [0.90424134, 0.18344086, 0.49853778, 0.55688786, 0.82617351],
#        [0.91290323, 0.24150538, 0.53341491, 0.62330962, 0.85674799],
#        [0.90357228, 0.20301075, 0.44538039, 0.57839154, 0.84580122]])
# best pos_weight = 2

array([[0.88677419, 0.12430108, 0.52165945, 0.47706149, 0.84656246],
       [0.88882915, 0.17010753, 0.50038028, 0.47980143, 0.83313419],
       [0.90169654, 0.19010753, 0.51369912, 0.5431016 , 0.8435737 ],
       [0.90663082, 0.18258065, 0.52858373, 0.57419763, 0.85126949],
       [0.90424134, 0.18344086, 0.49853778, 0.55688786, 0.82617351],
       [0.91290323, 0.24150538, 0.53341491, 0.62330962, 0.85674799],
       [0.90357228, 0.20301075, 0.44538039, 0.57839154, 0.84580122]])

In [27]:
x = merged.iloc[:, 7+class_dim+vec_dim:7+class_dim+vec_dim*2]

pos_weight = 2
cv_scores2 = np.empty((k_folds, num_metrics))
# Now, let's test the model with average vectors
j=0
for train_index, test_index in kf.split(x):
    print('{:d}th CV run'.format(j))
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Build model
    model = keras.models.Sequential() 
    model.add(keras.layers.Dense(128, input_dim=vec_dim, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(class_dim, activation='sigmoid')) # sigmoid so that the probability of one class is independent from the probability of another
    model.compile(loss=weighted_binary_crossentropy, 
              optimizer='adam')
    # Fit model
    model.fit(X_train, 
                 y_train,
                 epochs=epochs,
                 batch_size=batch_size,
                 verbose=0)

    # Evaluate
    y_prob = model.predict(X_test) # probabilities
    metrics = calc_metrics(y_test.values, y_prob)

    print(metrics)

    for k in range(num_metrics):
        cv_scores2[j, k] = metrics[k]
    j = j + 1

0th CV run
[0.9014336917562724, 0.06451612903225806, 0.4086021505376344, 0.5277777777777778, 0.7576426095178065]
1th CV run
[0.9014336917562724, 0.22580645161290322, 0.4329896907216495, 0.5, 0.7598409542743539]
2th CV run
[0.9014336917562724, 0.16129032258064516, 0.4444444444444445, 0.55, 0.7831255731802589]
3th CV run
[0.8944444444444445, 0.1, 0.3736263736263736, 0.5151515151515151, 0.7664901988839605]
4th CV run
[0.8814814814814815, 0.1, 0.3333333333333333, 0.5, 0.796875]


In [28]:
cv_scores2.mean(axis=0) 
# array([0.8960454 , 0.13032258, 0.3985992 , 0.51858586, 0.77279487])

array([0.8960454 , 0.13032258, 0.3985992 , 0.51858586, 0.77279487])